<a href="https://colab.research.google.com/github/Rashi2011/Rashi-Madhukar/blob/master/N_Layer_Model_on_Heart_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics


In [ ]:
#Loading the data()
#Downloading the dataset from local drive
from google.colab import files
uploaded = files.upload()


Saving Heart.csv to Heart.csv


In [ ]:
import io
dataset = pd.read_csv(io.BytesIO(uploaded['Heart.csv']))

In [ ]:
dataset = dataset.dropna()
X = dataset.iloc[:,:14].values
Y = dataset.iloc[:,14].values

#Encoding Categorical Data
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
lb = LabelEncoder()
X[:,3] = lb.fit_transform(X[:,3])
X[:,13] = lb.fit_transform(X[:,13])
#imputer(It takes 2d array)
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
X = imp.fit_transform(X)
Y = lb.fit_transform(Y)

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)




In [ ]:
#Splitting train and test set
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.3,random_state = 0)

#Reshaping X and Y(I/p and O/p)
X_train = np.transpose(X_train)
X_test = np.transpose(X_test)
Y_train= np.expand_dims(Y_train, axis=0)
Y_test= np.expand_dims(Y_test, axis=0)
print(X_train.shape)
print(Y_train.shape)

(14, 207)
(1, 207)


In [ ]:
#Initialize Parameters
def initialize_param(layers_dims):
  np.random.seed(102)
  parameters = {}                #empty dictionary
  L = len(layers_dims)           # no of layers in the network
  for l in range(1,L):
    parameters["W" + str(l)]= np.random.rand(layers_dims[l],layers_dims[l-1])
    parameters["b"+ str(l)]= np.zeros((layers_dims[l],1))

  return parameters

In [ ]:
'''
layers_dims = [14,5,3,1]
parameters = initialize_param(layers_dims)
#print(parameters)             #Dictionary
print("W1 = "+ str(parameters["W1"]))
print("b1 = "+ str(parameters["b1"]))
print("W2 = "+ str(parameters["W2"]))
print("b2 = "+ str(parameters["b2"]))
'''

'\nlayers_dims = [14,5,3,1]\nparameters = initialize_param(layers_dims)\n#print(parameters)             #Dictionary\nprint("W1 = "+ str(parameters["W1"]))\nprint("b1 = "+ str(parameters["b1"]))\nprint("W2 = "+ str(parameters["W2"]))\nprint("b2 = "+ str(parameters["b2"]))\n'

In [ ]:
#Activation Function
def sigmoid(z):
  A = 1/(1+np.exp(-z))
  return A

def relu(z):
  A = np.maximum(0,z)
  return A

def sign_f(AL,mean):
    if AL <= mean:
      return 0
    else:
      return 1
'''
def sigmoid_backward(dA,activation_cache):
  Z_ ,cacheee = sigmoid(dA)
  dZ = dA * Z_
  return dZ


def relu_backward(dA,activation_cache):
  Z_, cachee = relu(dA)
  dZ = dA *Z_
  return dZ
'''

'\ndef sigmoid_backward(dA,activation_cache):\n  Z_ ,cacheee = sigmoid(dA)\n  dZ = dA * Z_\n  return dZ\n\n\ndef relu_backward(dA,activation_cache):\n  Z_, cachee = relu(dA)\n  dZ = dA *Z_\n  return dZ\n'

In [ ]:
#forward
def forward(A, W, b):
  Z = np.dot(W,A) +b
  #print("Z.shape = ",Z.shape)       
  
  return Z

In [ ]:
#Activations forward
def activation_forward(A_prev, W, b ,activation):
  if activation == "sigmoid":
    #print("W,b",W.shape,b.shape)
    Z = forward(A_prev,W,b)
    A  = sigmoid(Z)
    

  elif activation == "relu":
    #print("W,b",W.shape,b.shape)
    Z = forward(A_prev,W,b)
    A  =  relu(Z)
    #print("Ashapes",A.shape)
          
  return A,Z

In [ ]:
#Forward Propagation
def forward_model(X,parameters):    
  cache = {}  #empty dictionary    
  A = X
  L = len(parameters)//2
 
  for l in range(1,L):
    A_prev =A
    A,cache["Z"+str(l)] = activation_forward(A_prev, parameters['W'+ str(l)], parameters['b'+ str(l)], activation = "relu")  #i = 1,i = 2
    cache["A"+str(l)] = A
    
    
  AL,cache["Z"+str(L)] = activation_forward(cache["A"+str(L-1)], parameters["W"+str(L)], parameters["b"+str(L)], activation = "sigmoid")  #i = 3
  cache["A"+str(L)] = AL
  return AL,cache



In [ ]:
#tocheck'''
'''
AL,cache = forward_model(X_train,parameters)
print(AL.shape)
'''

'\nAL,cache = forward_model(X_train,parameters)\nprint(AL.shape)\n'

In [ ]:
#Cost Function
def compute_cost(AL,Y):
  m = Y.shape[1]
  #print(m)
  cost = -(1/m)*np.sum((Y*np.log(AL) + (1-Y)*np.log(1-AL)))
  return cost

In [ ]:
#to check
'''
cost = compute_cost(AL,Y_train)
print("cost = ",cost)
'''

'\ncost = compute_cost(AL,Y_train)\nprint("cost = ",cost)\n'

In [ ]:
#Backward 
def backward(dA,W,A_prev): 
  m = dA.shape[1]            
  dW = np.dot(dA,A_prev.T)   #(1x3)
  dA_ = np.dot(W.T,dA)       #3x207
  #print("dA_",dA_.shape)
  db = (1/m)*np.sum(dA,axis = 1,keepdims = True) 

  return dA_,dW,db

In [ ]:
#Activation backward
def activation_backward(dA,W,A_prev ,activation):
  
  if activation == "relu":
    dA_,dW ,db = backward(dA,W,A_prev)
  elif activation == "sigmoid":
    dA_,dW ,db = backward(dA,W,A_prev)

  return dA_,dW,db



In [ ]:
#backward Model
def backward_model(AL,X,Y, cache,parameters):
  
  grads = {}           #empty dictionary
  L = int(len(parameters)/2)              #3    #no of layers  
  #print(L)  #no of layers
  m = Y.shape[1]                #no of training examples
  Y = Y.reshape(AL.shape)       #to ensure shape of Y = AL
  #intialization of backward propagation
  dAL = AL- Y           #dA3
  #print("dAL",dAL.shape)
  #Sigmoid (for layer L = L-1 )       #L = 3
  
  grads["dA"+ str(L-1)],grads["dW"+ str(L)] ,grads["db"+ str(L)]  = activation_backward(dAL,parameters["W"+str(L)],cache["A" + str(L-1)] ,activation = "sigmoid")

  # for layer (L =L-2 to  L=0 ) 
  for l in reversed(range(L-1)):    #L-1 = 2, #(0 to 2) # means l = 0,1 # reversing it  l = 1,0
    if l!=0:
      grads["dA"+ str(l)],grads["dW"+ str(l+1)] ,grads["db"+ str(l+1)]  = activation_backward(grads["dA"+ str(l+1)], parameters["W"+ str(l+1)],cache["A"+str(l)] ,activation = "relu")

    else:
      _,grads["dW"+ str(l+1)] ,grads["db"+ str(l+1)]  = activation_backward(grads["dA"+ str(l+1)], parameters["W"+ str(l+1)],X ,activation = "relu")
  return grads
     




In [ ]:
#To check
'''
grads = backward_model(AL,Y_train,cache,parameters)
print(grads)
'''

'\ngrads = backward_model(AL,Y_train,cache,parameters)\nprint(grads)\n'

In [ ]:
#Update Parameters
def update_param(parameters,grads,learning_rate):
  L = len(parameters)//2
  for l in range(L):
    parameters["W"+str(l+1)] = parameters["W"+str(l+1)] - (learning_rate*(grads["dW"+ str(l+1)]))
    parameters["b"+str(l+1)] = parameters["b"+str(l+1)] - (learning_rate*(grads["db"+ str(l+1)]))

  return parameters

In [ ]:

#To Check
'''
parameters = update_param(parameters,grads,learning_rate = 0.01)

print(parameters["W"+str(1)])
print(parameters["b"+str(1)])
'''

'\nparameters = update_param(parameters,grads,learning_rate = 0.01)\n\nprint(parameters["W"+str(1)])\nprint(parameters["b"+str(1)])\n'

In [ ]:
#NN Model
def L_layer_model(X, Y, layers_dims, learning_rate , num_iterations):
  costs = []                #empty list
  
  parameters = initialize_param(layers_dims)
  for i in range(0,num_iterations):
    #Forward Propagation(relu to sigmoid)
    
    AL,cache = forward_model(X,parameters)
    

    #compute cost
    cost = compute_cost(AL,Y)

    #Backward Propagation
    grads = backward_model(AL,X,Y, cache,parameters)

    #Update Parameters
    parameters = update_param(parameters,grads,learning_rate)
    

    #Print the cost on every 100th training examples
    
    print ("Cost after iteration %i: %f" %(i, cost))
    
    #Converting to y_pred
    
    y_pred = []
    for j in range(AL.shape[1]):
      y_pred.append(sign_f(AL[0][j],np.mean(AL)))
    y_pred = np.asmatrix(y_pred)
    
    costs.append(cost)
    #Accuracy
    print(metrics.f1_score(Y, y_pred,average = 'micro'))
    
         
  
  return parameters



In [ ]:
layers_dims = [14,5,1]
parameters = L_layer_model(X_train, Y_train, layers_dims, 0.005,num_iterations = 50)

Cost after iteration 0: 0.966128
0.6564102564102564
Cost after iteration 1: 0.650848
0.6923076923076923
Cost after iteration 2: 0.627131
0.7052023121387283
Cost after iteration 3: 0.635488
0.7182320441988952
Cost after iteration 4: 0.594914
0.7349397590361445
Cost after iteration 5: 0.596477
0.7403314917127073
Cost after iteration 6: 0.566455
0.7719298245614035
Cost after iteration 7: 0.560952
0.797752808988764
Cost after iteration 8: 0.544436
0.7885714285714286
Cost after iteration 9: 0.537636
0.7932960893854749
Cost after iteration 10: 0.529632
0.7954545454545454
Cost after iteration 11: 0.524304
0.8156424581005587
Cost after iteration 12: 0.518664
0.8268156424581005
Cost after iteration 13: 0.512996
0.8222222222222222
Cost after iteration 14: 0.506868
0.8156424581005587
Cost after iteration 15: 0.500227
0.8156424581005587
Cost after iteration 16: 0.492174
0.8222222222222222
Cost after iteration 17: 0.483802
0.8222222222222222
Cost after iteration 18: 0.474632
0.8222222222222222
Cost